In [1]:
!pwd

/Users/alex/Documents/Study/Mipt_OOP/Project


In [5]:
!python3 -m pip install simpletransformers

You should consider upgrading via the '/Users/alex/.env/bin/python3 -m pip install --upgrade pip' command.


In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup, RobertaTokenizer, RobertaModel
import torch
from torch import nn
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences as pad

Using TensorFlow backend.


In [43]:
def do_qa_test(test):
    output = []
    for line in test:
        context = line[1]
        qas = []
        question = line[-1]
        qid = line[0]
        if type(context) != str or type(question) != str:
            print(context, type(context))
            print(answer, type(answer))
            print(question, type(question))
            continue
        answers = []
        answers.append({'answer_start': 1000000, 'text': '__None__'})
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})
        output.append({'context': context.lower(), 'qas': qas})
    return output



In [8]:
from simpletransformers.question_answering import QuestionAnsweringModel

MODEL_PATH = '/Users/alex/Documents/Study/Mipt_OOP/Project/pretrainedDistilBERT/'

# Create the QuestionAnsweringModel
model = QuestionAnsweringModel('distilbert', 
                               MODEL_PATH, 
                               args={'reprocess_input_data': True,
                                     'overwrite_output_dir': True,
                                     'learning_rate': 5e-5,
                                     'num_train_epochs': 3,
                                     'max_seq_length': 192,
                                     'doc_stride': 64,
                                     'fp16': False,
                                    },
                              use_cuda=False)

#model.train_model('data/train.json')

In [23]:
pd.read_csv("train.csv").head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [12]:
data = np.asarray(pd.read_csv("train.csv"))

In [17]:
data_test = data[int(len(data)*0.2):]

In [19]:
data_test[0]

array(['4c3ce4cb16',
       ' u were doing good with the replys now u fallin off sad face',
       'u fallin off sad face', 'negative'], dtype=object)

In [24]:
answers = np.asarray(pd.read_csv("train.csv")['selected_text'])[int(len(data)*0.2):]

In [20]:
test = do_qa_test(data_test)

In [29]:
predictions = model.predict(test)

100%|██████████| 21985/21985 [00:08<00:00, 2693.45it/s]


In [35]:
predictions

[{'id': '4c3ce4cb16', 'answer': 'u fallin off sad face'},
 {'id': 'fde4e6a634', 'answer': 'about to dye my hair and do my nails'},
 {'id': 'd207933125',
  'answer': 'learning french funy school-day today don`t be mad about english :*:*'},
 {'id': 'd74fb2d0d9', 'answer': "dyed my hair. .'back to black'"},
 {'id': '8169cd1509', 'answer': 'making mother`s day gift'},
 {'id': '082b083743', 'answer': 'slow.'},
 {'id': '130c2fd973',
  'answer': 'me too! i don`t know why every time i listen to it i start to cry!'},
 {'id': 'a44666f258', 'answer': 'sick'},
 {'id': '3e0cca1c50', 'answer': "' wow hilarious"},
 {'id': '1986a93d8e', 'answer': 'missed my chance this time'},
 {'id': 'e647ca1ee1', 'answer': 'one of the best!'},
 {'id': '8e2cd98874', 'answer': 'sorry'},
 {'id': 'c2b3f9d6cf', 'answer': 'exciting'},
 {'id': 'b5922870f6', 'answer': 'yes, this nba song is great!!'},
 {'id': '691f395bee', 'answer': 'awesome!'},
 {'id': '035954f564', 'answer': 'love'},
 {'id': 'd90d309eb2', 'answer': 'that`

In [37]:
def evaluate(answers, predictions):
    jacc_score = 0.0
    for answer, prediction in zip(answers, predictions):
        jacc_score += 1-nltk.jaccard_distance(set(answer), set(prediction['answer']))
    
    return jacc_score/len(answers)

In [38]:
evaluate(answers, predictions)

0.8253505111478522

In [48]:
model = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model)
EPOCHS = 20
SPLIT = 0.8
MAXLEN = 48
DROP_RATE = 0.3
np.random.seed(42)

OUTPUT_UNITS = 3
BATCH_SIZE = 256
LR = (4e-5, 1e-2)
ROBERTA_UNITS = 768
VAL_BATCH_SIZE = 384
class Roberta(nn.Module):
    """
    An implementation of roBERTa model.
    It 
    """
    def __init__(self):
        super(Roberta, self).__init__()
        self.softmax = nn.Softmax(dim=1)
        self.drop = nn.Dropout(DROP_RATE)
        self.roberta = RobertaModel.from_pretrained(model)
        self.dense = nn.Linear(ROBERTA_UNITS, OUTPUT_UNITS)
        
    def forward(self, inp, att):
        inp = inp.view(-1, MAXLEN)
        _, self.feat = self.roberta(inp, att)
        return self.softmax(self.dense(self.drop(self.feat)))
    
    
class TweetDataset(torch.utils.data.Dataset):
    """
    Улучшенный датасет.
    Параметры:
    ::param data - датафрейм с данными
    ::param tokenizer - токенайзер, который будет применяться к тексту
    
    """
    def __init__(self, data, tokenizer):
        self.data = data
        self.text = data.text
        self.tokenizer = tokenizer
        self.sentiment = data.sentiment
        self.sentiment_dict = {"positive": 0, "neutral": 1, "negative": 2}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i:int) -> tuple:
        start, finish = 0, 2
        pg, tg = 'post', 'post'
        tweet = str(self.text[i]).strip()
        tweet_ids = self.tokenizer.encode(tweet)

        attention_mask_idx = len(tweet_ids) - 1
        if start not in tweet_ids: tweet_ids = start + tweet_ids
        tweet_ids = pad([tweet_ids], maxlen=MAXLEN, value=1, padding=pg, truncating=tg)

        attention_mask = np.zeros(MAXLEN)
        attention_mask[1:attention_mask_idx] = 1
        attention_mask = attention_mask.reshape((1, -1))
        if finish not in tweet_ids: tweet_ids[-1], attention_mask[-1] = finish, start
            
        sentiment = [self.sentiment_dict[self.sentiment[i]]]
        sentiment = torch.FloatTensor(to_categorical(sentiment, num_classes=3))
        return sentiment, torch.LongTensor(tweet_ids), torch.LongTensor(attention_mask)

def evaluate_roberta(network, dataset):
    dataloader = torch.utils.data.DataLoader(dataset)
    f_1 = []
    for batch in dataloader:
            targ, t_in, att = batch
            

            preds = np.asarray(network.forward(t_in, att).detach() > 0.5, dtype=np.int32)
            #print(preds)
            f_1.append( f1_score( targ.squeeze(dim=1).detach().numpy(), preds, average="weighted")/len(t_in))
            
            
    return np.mean(f_1)    
    
roberta = Roberta() 
ROBERTA_PATH = "/Users/alex/Downloads/sentiment_model.pt"
roberta.load_state_dict(torch.load(ROBERTA_PATH, map_location=torch.device('cpu')))

data_test = pd.read_csv("train.csv")[int(len(data)*0.2):]
data_test.index = np.arange(len(data_test))
data_test_ds = TweetDataset(data_test, tokenizer)
evaluate_roberta(roberta, data_test_ds)


/Users/alex/.env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.8238344325676598